# Множественная проверка гипотез: разбор кейса

А теперь разберём практический кейс. Он будет похож на задачу с поиском экстрасенсов в предыдущем юните.

У вас есть такие данные о результативности штрафных бросков игроков NBA:

СКАЧАТЬ ДАННЫЕ >>> https://drive.google.com/file/d/1kaGm1Y2M0pQBbL7TFkcnjyAw0niR0WoZ/view?usp=sharing

Каждый сезон NBA так же, как и в других видах спорта, проходит два этапа: регулярный сезон и плей-офф, когда играют на вылет. Понятно, что второй этап для всех более волнительный, и результативность игроков может отличаться. Мы попробуем проверить гипотезу о том, что этот этап сезона никак не влияет на результативность штрафных бросков игроков. И посчитаем разными методами количество игроков, у которых такое влияние всё-таки отмечается.

In [3]:
# Для начала нам надо посчитать данные:

import pandas as pd
import numpy as np

data = pd.read_csv('free_throws.csv')

In [4]:
# Посмотрим на формат:
data.head()

,end_result,game,game_id,period,play,player,playoffs,score,season,shot_made,time
0,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,0 - 1,2006 - 2007,1,11:45
1,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 2 of 2,Andrew Bynum,regular,0 - 2,2006 - 2007,1,11:45
2,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,1,7:26
3,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum misses free throw 2 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,0,7:26
4,106 - 114,PHX - LAL,261031013.0,1.0,Shawn Marion makes free throw 1 of 1,Shawn Marion,regular,21 - 12,2006 - 2007,1,7:18


In [5]:
data.playoffs.unique()

array(['regular', 'playoffs'], dtype=object)

Здесь нас интересуют следующие колонки:

- player — имя игрока
- playoffs — этап сезона NBA (regular/playoffs)
- shot_made — попал или не попал игрок штрафной бросок

Теперь будем для каждого игрока считать его среднюю результативность (долю попаданий) во время регулярного чемпионата и плей-офф, p_value, полученное через z-test, и запишем это в новый датафрейм. Z-test используем потому, что в данном случае наша задача является аналогом конверсии, например, в интернет-магазине, то есть мы имеем дело с распределением Бернулли.

Также мы оставим только тех игроков, у которых есть хотя бы по 30 бросков в регулярном сезоне и плей-офф. Это делаем для более корректной оценки, чтобы не считать тех игроков, у которых слишком мало бросков. Цифра 30 исходит из закона больших чисел, если коротко — такого количества наблюдений достаточно, чтобы довольно точно аппроксимировать нормальное распределение.

Так будет выглядеть код:

### ----- DECOMPOSITION OF CODE -----

In [29]:
from statsmodels.stats.weightstats import ztest

new_df = {
    "player": [],
    "regular_mean": [],
    "playoff_mean": [],
    "p-value": []
}

for player, group in data.groupby("player"):
    regular_shots = group[group["playoffs"] == "regular"]["shot_made"].values
    playoff_shots = group[group["playoffs"] == "playoffs"]["shot_made"].values

In [30]:
regular_shots

array([1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,

In [24]:
playoff_shots

array([0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1], dtype=int64)

### ----- FULL CODE -----

In [9]:
from statsmodels.stats.weightstats import ztest

new_df = {
    "player": [],
    "regular_mean": [],
    "playoff_mean": [],
    "p_value": []
}

for player, group in data.groupby("player"):
    regular_shots = group[group["playoffs"] == "regular"]["shot_made"].values
    playoff_shots = group[group["playoffs"] == "playoffs"]["shot_made"].values
    
    if len(regular_shots) < 30 or len(playoff_shots) < 30:
        continue
    
    statistic, p_value = ztest(regular_shots, playoff_shots)
    
    new_df["player"].append(player)
    new_df["regular_mean"].append(np.mean(regular_shots))
    new_df["playoff_mean"].append(np.mean(playoff_shots))
    new_df["p_value"].append(p_value)
    
new_df = pd.DataFrame(new_df)

In [19]:
# Посмотрим на 10 случайных элементов, чтобы увидеть, что у нас получилось:

new_df.sample(10)

,player,regular_mean,playoff_mean,p_value
197,P.J. Brown,0.779412,0.791667,0.860278
124,Joel Anthony,0.662469,0.758065,0.134794
106,James Jones,0.843091,0.888889,0.289811
233,Shaquille O'Neal,0.523077,0.518248,0.913809
11,Andre Miller,0.808634,0.768786,0.201764
170,Marcus Smart,0.721649,0.694444,0.732758
161,Louis Williams,0.826116,0.812500,0.778761
28,Boris Diaw,0.720817,0.700000,0.644653
105,James Harden,0.854720,0.867021,0.435286
230,Serge Ibaka,0.744828,0.768293,0.520975


##### Теперь посчитаем количество игроков с p_value < 0.05. Так мы поймём, для скольких игроков мы бы отвергли нулевую гипотезу без поправок о множественной проверки гипотез.

Смотрим:

In [22]:
new_df[new_df.p_value <= 0.05].shape

(22, 4)

##### Расчёты готовы, и нам необходимо сравнить их с методами, изученными в предыдущем юните. Для начала импортируем функцию для множественной проверки гипотез:

In [23]:
from statsmodels.stats.multitest import multipletests

Cравним результаты 
##### с поправкой Бонферрони и методом Холма:

In [26]:
multipletests(new_df.p_value, alpha = 0.05, method = 'bonferroni')[0].sum()

1

In [28]:
multipletests(new_df.p_value, alpha = 0.05, method = 'holm')[0].sum()

1

#### Видим, что в данной задаче два метода выдали одинаковое количество игроков, для которых стоит отвергнуть нулевую гипотезу. То есть тех, у кого с 95% отличается результативность во время плей-офф и регулярного сезона.

#### Однако мы видим, насколько данное число отличается от того, что мы получили без поправок на множественную проверку гипотез. Так нам удалось кратно уменьшить вероятность ошибок первого рода.